##QUnoJenga

We were arguing for a lot, so we decided to implement both games (: 

##Auth to Azure and imports

In [1]:
!pip install azure-quantum --quiet --no-warn-conflicts
!pip install azure-quantum[qiskit] --quiet

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, Aer, transpile
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_histogram
from qiskit.circuit.random import random_circuit
from qiskit.tools.monitor import job_monitor
import random
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
from azure.quantum.qiskit import AzureQuantumProvider

key_path = "/content/drive/MyDrive/Colab Notebooks/quantum111/key.txt"
if os.path.isfile(key_path):
    with open(key_path, 'r') as file:
        key = file.read().replace('\n', '')
else:
    key = input("Enter your Azure Resource ID").lower()

provider = AzureQuantumProvider(
    resource_id = key
    location = "eastus"
)

In [5]:
provider.backends()

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RY737JT8F to authenticate.


[<IonQQPUBackend('ionq.qpu')>, <IonQSimulatorBackend('ionq.simulator')>]

##Imports

#Game classes definitions

Qubits number


In [6]:
n = 5

In [7]:
class Card:
    def __init__(self, number, color, quantum_circuit):
        self.number = number
        self.color = color
        self.qc = quantum_circuit
    def draw(self, draw_qc=False):
        print('////////////////////////')
        print(f'COLOR: {self.color}')
        print(f'NUMBER: {self.number}')
        if draw_qc:
            print(self.qc.draw())
        print('////////////////////////')
        

In [8]:
class CollectionOfCards:
    def __init__(self):
        self.collection = []

    def add_card(self, card):
        self.collection.append(card)
      
    def take_card(self):
        
        #new deck if there are no cards left
        if self.get_number_of_cards() == 0:
            self.make_deck()

        #take the last card
        card = self.collection[-1]
        self.collection.remove(card)
        return card

    def make_deck(self):
        for num in range(1, 10):
            for color in ["Yellow", "Red", "Blue", "Green"]:
                random_qc = random_circuit(n, depth=num, max_operands=2, seed=42)   #make random  
                newcard = Card(num, color, random_qc)
                self.add_card(newcard)

    def get_number_of_cards(self):
        return len(self.collection)

    def get_top_card(self):
        return self.collection[-1]

    def can_play(self, new_card):
        top_card = self.get_top_card()
        if (top_card.number == new_card.number) or (top_card.color == new_card.color):
            return True
        return True#False

In [9]:
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = CollectionOfCards()

    def get_number_of_cards(self):
        return self.hand.get_number_of_cards()

    def look_card(self, i):
        if i not in list(range(len(self.hand.collection))):
            print('Wrong card number')
        else:
            card = self.hand.collection[i]
            card.draw()
    
    def look_all_cards(self):
        for i in range(len(self.hand.collection)):
            print()
            print(f'CARD {i}')
            self.look_card(i)
            print()

#QUnoJenga description

In [16]:
class QUnoJenga:
    def __init__(self, n_of_players=2):
        self.deck = CollectionOfCards()
        self.deck.make_deck()
        random.shuffle(self.deck.collection)
        
        self.players = [Player('Artem'), Player('Leyla')]
        self.current_player_number = 0
        self.winner_number = -1
        
        for i in range(3):
            for player in self.players:
                player.hand.add_card(self.deck.take_card())


        self.center = CollectionOfCards()
        self.center.add_card(self.deck.take_card())


    def play_turn(self, player):

        turn_done = False
        while(turn_done == False):
            #Info for user
            print()
            print(f'{player.name}, your turn!')
            print(f'You have {player.hand.get_number_of_cards()} cards')
            print(f'The center card is {self.center.get_top_card().number}, {self.center.get_top_card().color}. ')
            print(f'Select your option')

            options_list = ['look card', 'look all cards', 'play card', 'skip', 'REAL HARDWARE']
            options_dictionary = {i: options_list[i] for i in range(len(options_list))}

            for key,value in options_dictionary.items():
                print(key, '--', value)

            #Select option and do corresponding action
            selected_option_number = int(input())

            if options_dictionary[selected_option_number] == 'look card':
                card_number = int(input(f'{player.name}, enter the number of card to look'))
                player.look_card(card_number)

            if options_dictionary[selected_option_number] == 'look all cards':
                player.look_all_cards()

            if options_dictionary[selected_option_number] == 'play card':
                card_number = int(input(f'{player.name}, enter the number of card to play'))

                if card_number not in list(range(len(player.hand.collection))):
                    print('Wrong card number')
                else:
                    card = player.hand.collection[card_number]
                    card.draw()
                    if self.center.can_play(card):
                        player.hand.collection.remove(card)
                        self.center.add_card(card)
                        turn_done = True
                    else:
                        print('You cannot play this card. Check the color and the number of the center card')
            
            if options_dictionary[selected_option_number] == 'skip':
                turn_done = True
            if options_dictionary[selected_option_number] == 'REAL HARDWARE':
                print('Comparing classical and quantum hardware')
                turn_done = True
                self.winner_number = self.evaluate()

        print('Turn done')
        print()

    def play_game(self):

        while(True):
            
            current_player = self.players[self.current_player_number]

            self.play_turn(current_player)
            
            if self.winner_number != -1:
                  winner = self.players[self.winner_number]
                  print(f"{winner.name} is winner")
                  break

            self.current_player_number = (self.current_player_number + 1 ) % len(self.players)
    
        
    def evaluate(self):

        simulator_backend = provider.get_backend("ionq.simulator")
        real_backend = provider.get_backend("ionq.qpu")

        circuit = QuantumCircuit(n, n)
        return random.randint(0, len(self.players) - 1)

#Evaluate on quantum and classical hardware

In [97]:
def generate_noise_qc(qubits=5):
    """
    Generates a circuit that can be used to simulate noise. 
    Intended to be used between the cards when playing without the real machine.
    No measurement at the end.
    """
    ph_noise = 0.1
    noise_qc = QuantumCircuit(qubits, qubits)
  
    for q in range(qubits):
      # Generate random rotations in all 3 directions for every qubit
        noise_qc.rx(random.gauss(0,ph_noise), q)
        noise_qc.ry(random.gauss(0,ph_noise), q)
        noise_qc.rz(random.gauss(0,ph_noise), q)
    return noise_qc


def run_qc(qc, backend, shots):
    """
    Run quantum circuit on backend
    """
    #circ_backend = provider.get_backend(backend)
    circ_backend = Aer.get_backend('aer_simulator')
    job = circ_backend.run(transpile(qc, circ_backend), shots=shots)
    print(f"Job id on backend {circ_backend}")

    return job.result()


def counts_to_prob(counts, qubits=5):
    """
    Converts dict of counts to dict of probabilites
    """
    print(counts)

    # value to recalculate counts to wavefunction amplitudes
    norm = sum(counts.values())

    # dict for probabilities
    probs = {}

    # loop over all dict values, missing ones are = 0
    for i in range(2 ** qubits):
      key = format(i,'05b')
      if key in counts:
        count = counts[key]
      else:
        count = 0
      probs[key] = np.sqrt(count / norm)
      

    return probs


def overlap(counts_s, counts_q, qubits=5):
    """
    Calculates the overlap between two distributions as (sum(c1i*c2i))^2
    """

    # convert counts to probabilities
    probs1 = counts_to_prob(counts_s)
    probs2 = counts_to_prob(counts_q)
      
    overlap = 0
    # loop over all dict values, no ones should be missing
    for i in range(2 ** qubits):
        key = format(i,'05b')
        c1 = probs1[key]
        c2 = probs2[key]
        overlap += c1 * c2

    return overlap
  
def compose_circuits(n, circuits, add_noise=False):
    """
    Create composite circuit from n first circuits from the list
    """
    circ = QuantumCircuit(5, 5)
    for i in range(n):
        circ = circ.compose(circuits[i])
        if add_noise:
            circ = circ.compose(generate_noise_qc(qubits=5))

    circ.measure_all()
    circ.draw()
    return circ


In [98]:
def evaluate(circuits, real_device=True):
    """
    Return set of overlaps by running on simulator and real device
    Also we have option run noise circuit on simulator instead of run on real device
    """

    result_overlap = []
    shots = 100

    for i in range(1, len(circuits)):
        ideal_qc = QuantumCircuit(5, 5)
        ideal_qc = compose_circuits(i, circuits, add_noise=False)

        # simulate noise circ
        qc_noised = QuantumCircuit(5, 5)
        if real_device == False:
            qc_noised = compose_circuits(i, circuits, add_noise=True)
        
        #run on sim
        result_s = run_qc(ideal_qc, backend='ionq.simulator', shots=shots)
        counts_s = {format(n, "05b"): 0 for n in range(2 ** 5)}
        print(counts_s)
        counts_s.update(result_s.get_counts(ideal_qc))
        print(counts_s)

        # run noise circ
        if real_device == False:
            result_q = run_qc(qc_noised, backend='ionq.simulator', shots=shots)
        else:
            result_q = run_qc(qc_noised, backend='ionq.qpu', shots=shots)

        counts_q = {format(n, "05b"): 0 for n in range(2 ** 5)}


        if real_device == False:
            counts_q.update(result_q.get_counts(qc_noised))
        else:
            counts_q.update(result_q.get_counts(qc_noised))

        result_overlap.append(overlap(counts_s, counts_q))

    return result_overlap

In [100]:
circuits = []
num_cards = 10 # len(circuits)

for i in range(num_cards):
    num = random.randint(1,9)
    circuits.append(random_circuit(n, num, max_operands=2, seed=42))
    # circuits.append(generate_circuit(num, n))

results = evaluate(circuits, real_device=False)
results

Job id on backend aer_simulator
{'00000': 0, '00001': 0, '00010': 0, '00011': 0, '00100': 0, '00101': 0, '00110': 0, '00111': 0, '01000': 0, '01001': 0, '01010': 0, '01011': 0, '01100': 0, '01101': 0, '01110': 0, '01111': 0, '10000': 0, '10001': 0, '10010': 0, '10011': 0, '10100': 0, '10101': 0, '10110': 0, '10111': 0, '11000': 0, '11001': 0, '11010': 0, '11011': 0, '11100': 0, '11101': 0, '11110': 0, '11111': 0}
{'00000': 0, '00001': 0, '00010': 0, '00011': 0, '00100': 0, '00101': 0, '00110': 0, '00111': 0, '01000': 0, '01001': 0, '01010': 0, '01011': 0, '01100': 0, '01101': 0, '01110': 0, '01111': 0, '10000': 0, '10001': 0, '10010': 0, '10011': 0, '10100': 0, '10101': 0, '10110': 0, '10111': 0, '11000': 0, '11001': 0, '11010': 0, '11011': 0, '11100': 0, '11101': 0, '11110': 0, '11111': 0, '00000 00000': 6, '01010 00000': 2, '01100 00000': 6, '01001 00000': 7, '00011 00000': 4, '01011 00000': 7, '01000 00000': 6, '00010 00000': 7, '00110 00000': 4, '00001 00000': 7, '01110 00000': 1, 

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

#GAME

In [17]:
my_game = QUnoJenga()
my_game.play_game()


Artem, your turn!
You have 3 cards
The center card is 1, Yellow. 
Select your option
0 -- look card
1 -- look all cards
2 -- play card
3 -- skip
4 -- REAL HARDWARE
4
Comparing classical and quantum hardware
Turn done

Artem is winner
